In [48]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [154]:
zonasGDF = gpd.read_file('files/input/ESTADI_Pol_ZonNav2000.shp')  
poblacionesGDF = gpd.read_file('files/input/ESTADI_Pol_EntidadPob.shp')  

In [174]:
zonasGDF

,FEATURE,CZONA,ZONA,GEOM_AREA,GEOM_PERI,BEGINLIFE,geometry
0,5000003,1,Noroeste,1.897209e+09,326276.47,01/04/2003,"POLYGON ((622475.075 4767366.700, 622470.302 4..."
1,5000003,2,Pirineo,2.317741e+09,290628.21,01/04/2003,"POLYGON ((640467.528 4775412.001, 640488.367 4..."
2,5000003,3,Pamplona,7.778968e+08,170927.17,01/04/2003,"POLYGON ((588331.127 4746537.496, 588333.254 4..."
3,5000003,4,Tierra Estella,1.541949e+09,271158.21,01/04/2003,"POLYGON ((542696.015 4718265.244, 542704.095 4..."
4,5000003,5,Navarra Media Oriental,1.302106e+09,254505.44,01/04/2003,"MULTIPOLYGON (((591166.343 4715367.175, 591155..."
5,5000003,6,Ribera Alta,1.225675e+09,279483.88,01/04/2003,"POLYGON ((568830.804 4704339.051, 568825.875 4..."
6,5000003,7,Tudela,1.328951e+09,202715.10,01/04/2003,"POLYGON ((591193.134 4659198.195, 591203.088 4..."


In [254]:
poblacionesGDF.geometry = poblacionesGDF.centroid 
poblacionesGDF['ZONA'] = [zonasGDF.ZONA[zonasGDF.contains(geo)].values[0] for key, geo in poblacionesGDF.geometry.items()]
poblacionesGDF.to_file('files/output/poblaciones.shp')  
poblacionesGDF.head()

,FEATURE,CIDENTIDAD,CENTIDAD,ENTIDAD,CMUNICIPIO,MUNICIPIO,CATEGORIA,CORIGEN,ALTMEDIA,HABIT2003,...,HABIT2018,HABIT2019,HABIT2020,MUNINOAC,ENTINOAC,GEOM_AREA,GEOM_PERI,BEGINLIFE,geometry,ZONA
0,5000006,10001,1,Abáigar,1,Abáigar,Lugar/Tokia,1,491.0,97.0,...,87,83,82,Abaigar,Abaigar,93328.61,2034.94,15/02/2021,POINT (570293.561 4722182.916),Tierra Estella
1,5000006,20001,1,Abárzuza / Abartzuza,2,Abárzuza / Abartzuza,Lugar/Tokia,1,561.0,466.0,...,543,531,537,Abarzuza / Abartzuza,Abarzuza / Abartzuza,294540.36,5238.89,15/02/2021,POINT (580158.200 4730963.566),Tierra Estella
2,5000006,20002,2,Andéraz,2,Abárzuza / Abartzuza,Colegio/Ikastetxea,1,551.0,27.0,...,0,0,0,Abarzuza / Abartzuza,Anderaz,20434.79,708.94,15/02/2021,POINT (580972.565 4731209.283),Tierra Estella
3,5000006,20003,3,Irantzu,2,Abárzuza / Abartzuza,Monasterio/Monasterioa,1,727.0,17.0,...,7,7,7,Abarzuza / Abartzuza,Irantzu,17102.65,519.23,15/02/2021,POINT (578667.022 4733618.762),Tierra Estella
4,5000006,30001,1,Abaurregaina / Abaurrea Alta,3,Abaurregaina / Abaurrea Alta,Lugar/Tokia,1,1046.0,160.0,...,121,126,127,Abaurregaina/Abaurrea Alta,Abaurregaina/Abaurrea Alta,157916.23,2133.14,15/02/2021,POINT (646355.096 4751631.985),Pirineo


In [ ]:
filtered = filter(lambda col: col.find("HABIT") == 0, list(poblacionesGDF.columns))
cols_habitants = list(filtered)

In [259]:
centro_medio_points = {
    'geometry': [],
    'year': [],
    'zona': []
}

for i, zona in enumerate(zonasGDF.ZONA.values):
    poblacionesZonaGDF = poblacionesGDF.loc[poblacionesGDF.ZONA == zona]
    poblacionesZonaGDF = poblacionesZonaGDF.reset_index()

    for col_hab in cols_habitants:
        centro_medio = np.array([geo.coords * poblacionesZonaGDF[col_hab].iloc[key] / poblacionesZonaGDF[col_hab].sum() for key, geo in poblacionesZonaGDF.geometry.items()]).sum(axis = 0)
        if not np.isnan(centro_medio).any():
            centro_medio_points['geometry'].append(Point(*centro_medio))
            centro_medio_points['year'].append(col_hab)
            centro_medio_points['zona'].append(zona)
    
centroHabitantesZonasGDF = gpd.GeoDataFrame(centro_medio_points)  
if (centroHabitantesZonasGDF.crs is None):
    centroHabitantesZonasGDF = centroHabitantesZonasGDF.set_crs('epsg:4326')
centroHabitantesZonasGDF.to_file('files/output/centros_medios.shp')  

In [ ]:
centroHabitantesZonasGDF